In [23]:
import numpy as np
import pandas as pd

In [22]:
#歸一化
#name=""
name="_maxmin"

計算每天的缺失值，對48個timestep以下的缺失值進行插補

In [24]:
final_result = pd.read_csv(f'./data/preprocess_data/TimeMinGroupData{name}.csv')
final_result_1 = final_result.copy()
final_result_1.head(3)

,LocationCode,Date,TimeMinGroup,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01,07:00,0.637779,0.259197,0.853111,0.003829,0.000032
1,1,2024-01-01,07:10,0.638258,0.260368,0.852885,0.005433,0.000053
2,1,2024-01-01,07:20,0.639869,0.261371,0.843965,0.007386,0.000094


In [25]:
# 計算每個 LocationCode 和 Date 的缺失值行數
missing_count_per_group = final_result.groupby(['LocationCode', 'Date']).apply(
    lambda group: group.isnull().any(axis=1).sum()  # 計算該組中至少有一個缺失值的行數
).reset_index(name='MissingRowsCount')

# 找出需要刪除的組合（缺失值行數大於等於 48 的組合）
groups_to_delete = missing_count_per_group[missing_count_per_group['MissingRowsCount'] >= 48][['LocationCode', 'Date']]

# 過濾原始 DataFrame，刪除這些組合的資料
filtered_result = final_result.merge(groups_to_delete, on=['LocationCode', 'Date'], how='outer', indicator=True)
filtered_result = filtered_result[filtered_result['_merge'] == 'left_only'].drop(columns=['_merge'])

filtered_result.shape


/tmp/ipykernel_3035796/1236749476.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_count_per_group = final_result.groupby(['LocationCode', 'Date']).apply(


(107160, 8)

In [26]:
#列出missing_count_per_group中MissingRowsCount各數值的個數，並照數值大小排序
missing_count_per_group['MissingRowsCount'].value_counts().sort_index()
#missing_count_per_group[(missing_count_per_group['MissingRowsCount'] < 48) & (missing_count_per_group['MissingRowsCount'] != 0)]

MissingRowsCount
0     1535
1       25
2        7
3        2
4        6
5       10
6        8
7        7
8        5
9        4
10       5
11      13
12      10
13       5
14       6
15       1
16       2
17      11
18       3
19       3
20       2
21       1
22       7
23       5
24       5
25       1
26       9
27       8
28       5
29       3
30       5
31       3
32       3
34       5
35       5
36       2
37       7
38       3
39       7
40       5
42       6
43       3
44       6
45       6
46       2
47       4
48     224
49       3
50       6
51       3
52       7
53       5
54      10
55       4
56       1
57      13
58       7
59      13
Name: count, dtype: int64

In [27]:
# 對每個 LocationCode 和 Date 組合進行缺失值插補
interpolated_filtered_result = (
    filtered_result
    .groupby(['LocationCode', 'Date'], group_keys=False)  # 按 LocationCode 和 Date 分組
    .apply(lambda group: group.sort_values(by='TimeMinGroup').interpolate(method='linear', limit_direction='both'))
)

interpolated_filtered_result.head(3)


/tmp/ipykernel_3035796/3358108474.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.sort_values(by='TimeMinGroup').interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_3035796/3358108474.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.sort_values(by='TimeMinGroup').interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_3035796/3358108474.py:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda group: group.sort_values(by='TimeMinGroup').interpolate(method='linear', limit_direction='both'))
/tmp/ipykernel_3035796/33581

,LocationCode,Date,TimeMinGroup,Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,1,2024-01-01,07:00,0.637779,0.259197,0.853111,0.003829,0.000032
1,1,2024-01-01,07:10,0.638258,0.260368,0.852885,0.005433,0.000053
2,1,2024-01-01,07:20,0.639869,0.261371,0.843965,0.007386,0.000094


以插補值取代原資料

In [28]:
#把final_result_1中LocationCode、Date、TimeMinGroup組合有出現在interpolated_filtered_result中的，用interpolated_filtered_result的值取代final_result_1的值

# 定義要匹配的列
key_columns = ['LocationCode', 'Date', 'TimeMinGroup']

# 將 `interpolated_filtered_result` 的值替換到 `final_result_1` 中
# 使用 merge 確定重複鍵
merged_result = pd.merge(
    final_result_1,
    interpolated_filtered_result,
    on=key_columns,
    how='left',
    suffixes=('', '_new')
)

# 使用 `_new` 結尾的列替換原有值（假設所有替換的列都存在於 interpolated_filtered_result 中）
for col in interpolated_filtered_result.columns:
    if col not in key_columns:  # 避免覆蓋鍵值列
        merged_result[col] = merged_result[f"{col}_new"].combine_first(merged_result[col])

# 移除臨時列
merged_result = merged_result.drop(columns=[col for col in merged_result.columns if col.endswith('_new')])

merged_result.shape

(124920, 8)

刪除每天大於48缺失值的資料，缺48timestep以上的有72天，共72*10*6=4320row，124920-4320=120600

In [29]:

# 計算每個 LocationCode 和 Date 的缺失值行數
missing_count_per_group = merged_result.groupby(['LocationCode', 'Date']).apply(
    lambda group: group.isnull().any(axis=1).sum()  # 計算該組中至少有一個缺失值的行數
).reset_index(name='MissingRowsCount')

# 找出需要刪除的組合（缺失值行數大於 48 的組合）
groups_to_delete = missing_count_per_group[missing_count_per_group['MissingRowsCount'] > 48][['LocationCode', 'Date']]

# 過濾原始 DataFrame，刪除這些組合的資料
filtered_result = merged_result.merge(groups_to_delete, on=['LocationCode', 'Date'], how='outer', indicator=True)
filtered_result = filtered_result[filtered_result['_merge'] == 'left_only'].drop(columns=['_merge'])

filtered_result.shape


/tmp/ipykernel_3035796/3468898555.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_count_per_group = merged_result.groupby(['LocationCode', 'Date']).apply(


(120600, 8)

補足連續'Date'、'TimeMinGroup'

In [30]:
# 將 Date 列轉換為 datetime 類型
filtered_result['Date'] = pd.to_datetime(filtered_result['Date'])

# 取得唯一的 TimeMinGroup 值列表
time_min_groups = filtered_result['TimeMinGroup'].unique()

# 初始化一個空的 DataFrame 用來存放擴展後的數據
expanded_df = pd.DataFrame()

# 對每個 LocationCode 生成從最小到最大的日期範圍
for location in filtered_result['LocationCode'].unique():
    location_data = filtered_result[filtered_result['LocationCode'] == location]
    min_date = location_data['Date'].min()
    max_date = location_data['Date'].max()
    
    # 生成完整的日期範圍
    full_dates = pd.date_range(start=min_date, end=max_date, freq='D')
    
    # 創建每個日期和 TimeMinGroup 的多重索引
    location_df = pd.DataFrame(
        index=pd.MultiIndex.from_product([full_dates, time_min_groups], names=['Date', 'TimeMinGroup'])
    ).reset_index()
    
    # 指定 LocationCode
    location_df['LocationCode'] = location
    
    # 將原始數據合併進去，保留已有數據，缺失的部分以 NaN 補充
    location_df = location_df.merge(
        location_data,
        on=['LocationCode', 'Date', 'TimeMinGroup'],
        how='left'
    )
    
    # 將結果添加到擴展 DataFrame 中
    expanded_df = pd.concat([expanded_df, location_df], ignore_index=True)

# 重新排序欄位順序以符合原始結構
expanded_df = expanded_df[['LocationCode', 'Date', 'TimeMinGroup', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']]

#將expanded_df存成csv檔
expanded_df.to_csv(f'./data/preprocess_data/TimeMinGroupData_FullDate{name}_interpolation.csv',index=False)

針對每一個'LocationCode'，從'Date'中的最小值開始到最大值，將每一天的每一個'TimeMinGroup'的'Power(mW)'重新整理成每個row都是一筆連續兩天每個時段的值

In [31]:

# 取得 TimeMinGroup 排序
time_min_groups_sorted = sorted(expanded_df['TimeMinGroup'].unique())
columns_day_1 = [f"{time}_1_Power(mW)" for time in time_min_groups_sorted]
columns_day_2 = [f"{time}_2_Power(mW)" for time in time_min_groups_sorted]

# 初始化一個新的 DataFrame 來存儲重整後的數據
reshaped_data = []

# 處理每個 LocationCode
for location in expanded_df['LocationCode'].unique():
    location_data = expanded_df[expanded_df['LocationCode'] == location].sort_values(by=['Date', 'TimeMinGroup'])
    unique_dates = sorted(location_data['Date'].unique())
    
    # 迴圈處理每一對連續的兩天
    for i in range(len(unique_dates) - 1):
        day_1 = unique_dates[i]
        day_2 = unique_dates[i + 1]
        
        # 過濾出兩天的數據
        day_1_data = location_data[location_data['Date'] == day_1].set_index('TimeMinGroup')['Power(mW)']
        day_2_data = location_data[location_data['Date'] == day_2].set_index('TimeMinGroup')['Power(mW)']
        
        # 建立一行包含兩天的資料
        combined_row = {
            'LocationCode': location,
            'Date_Day1': day_1,
            'Date_Day2': day_2
        }
        
        # 將每個時間段的資料加入
        combined_row.update({col: day_1_data.get(time, np.nan) for col, time in zip(columns_day_1, time_min_groups_sorted)})
        combined_row.update({col: day_2_data.get(time, np.nan) for col, time in zip(columns_day_2, time_min_groups_sorted)})
        
        # 將每一行的資料加入到列表中
        reshaped_data.append(combined_row)

# 使用 pd.concat 來轉換為 DataFrame
reshaped_data_df = pd.DataFrame(reshaped_data)


In [32]:
#將reshaped_data_df有缺失值的row列出，並存成missing_data
missing_data = reshaped_data_df[reshaped_data_df.isnull().any(axis=1)]
missing_data.shape

missing_data.to_csv(f'./data/preprocess_data/PowerData{name}_na_interpolation.csv',index=False)

In [33]:
#將reshaped_data_df中有缺失值的row刪除，並存成csv檔
reshaped_data_df.dropna(inplace=True)
reshaped_data_df.shape

reshaped_data_df.to_csv(f'./data/preprocess_data/PowerData{name}_interpolation.csv',index=False)

將PowerData{name}_na_interpolation整理成可以當測試集的格式

In [34]:
data_na = pd.read_csv(f'./data/preprocess_data/PowerData{name}_na_interpolation.csv')

#把data_na中col4-col75欄位為nan的資料刪除
data_na = data_na.dropna(subset=data_na.columns[3:75])

#把data_na中16:10_2_Power(mW)欄位是nan的資料列出，並叫data_na_1
data_na_1 = data_na[data_na['16:10_2_Power(mW)'].isnull()]

#取data_na_1_onehot前75欄，後48欄刪除
data_na_1_onehot = data_na_1.iloc[:, 0:75]
print("Feature count of new data:", data_na_1_onehot.shape)

Feature count of new data: (204, 75)


In [35]:
# 檢查欄位名稱
column_names = data_na_1_onehot.columns.tolist()
print(column_names)

data_na_1_onehot.to_csv(f'./data/preprocess_data/PowerData{name}_na_interpolation_testset_1.csv', index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1_Power(mW)', '07:10_1_Power(mW)', '07:20_1_Power(mW)', '07:30_1_Power(mW)', '07:40_1_Power(mW)', '07:50_1_Power(mW)', '08:00_1_Power(mW)', '08:10_1_Power(mW)', '08:20_1_Power(mW)', '08:30_1_Power(mW)', '08:40_1_Power(mW)', '08:50_1_Power(mW)', '09:00_1_Power(mW)', '09:10_1_Power(mW)', '09:20_1_Power(mW)', '09:30_1_Power(mW)', '09:40_1_Power(mW)', '09:50_1_Power(mW)', '10:00_1_Power(mW)', '10:10_1_Power(mW)', '10:20_1_Power(mW)', '10:30_1_Power(mW)', '10:40_1_Power(mW)', '10:50_1_Power(mW)', '11:00_1_Power(mW)', '11:10_1_Power(mW)', '11:20_1_Power(mW)', '11:30_1_Power(mW)', '11:40_1_Power(mW)', '11:50_1_Power(mW)', '12:00_1_Power(mW)', '12:10_1_Power(mW)', '12:20_1_Power(mW)', '12:30_1_Power(mW)', '12:40_1_Power(mW)', '12:50_1_Power(mW)', '13:00_1_Power(mW)', '13:10_1_Power(mW)', '13:20_1_Power(mW)', '13:30_1_Power(mW)', '13:40_1_Power(mW)', '13:50_1_Power(mW)', '14:00_1_Power(mW)', '14:10_1_Power(mW)', '14:20_1_Power(mW)', '14:30_1_Pow

針對每一個'LocationCode'，從'Date'中的最小值開始到最大值，將每一天的每一個'TimeMinGroup'的'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', Sunlight(Lux), 'Power(mW)'重新整理成每個row都是一筆連續兩天每個時段的值

In [36]:
#按時間排列變數欄位（時間優先），即每個時間點包含四個變數，並按照時間點順序排列

# 定義要處理的變數
variables = ['Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']

# 取得 TimeMinGroup 排序並生成時間優先的欄位名稱
time_min_groups_sorted = sorted(expanded_df['TimeMinGroup'].unique())
columns_day_1 = [f"{time}_1_{var}" for time in time_min_groups_sorted for var in variables]
columns_day_2 = [f"{time}_2_{var}" for time in time_min_groups_sorted for var in variables]
all_columns = columns_day_1 + columns_day_2  # Day 1 欄位接 Day 2 欄位

# 初始化一個新的列表來存儲重整後的數據
reshaped_data_all = []

# 處理每個 LocationCode
for location in expanded_df['LocationCode'].unique():
    location_data = expanded_df[expanded_df['LocationCode'] == location].sort_values(by=['Date', 'TimeMinGroup'])
    unique_dates = sorted(location_data['Date'].unique())
    
    # 迴圈處理每一對連續的兩天
    for i in range(len(unique_dates) - 1):
        day_1 = unique_dates[i]
        day_2 = unique_dates[i + 1]
        
        # 建立一行包含兩天的資料
        combined_row = {
            'LocationCode': location,
            'Date_Day1': day_1,
            'Date_Day2': day_2
        }
        
        # 逐個時間段及變數加入數據
        for time in time_min_groups_sorted:
            day_1_data = location_data[location_data['Date'] == day_1].set_index('TimeMinGroup')
            day_2_data = location_data[location_data['Date'] == day_2].set_index('TimeMinGroup')
            
            combined_row.update({f"{time}_1_{var}": day_1_data[var].get(time, np.nan) for var in variables})
            combined_row.update({f"{time}_2_{var}": day_2_data[var].get(time, np.nan) for var in variables})
        
        # 添加到列表中
        reshaped_data_all.append(combined_row)

# 使用指定的欄位順序轉換為 DataFrame
reshaped_data_df_all = pd.DataFrame(reshaped_data_all, columns=['LocationCode', 'Date_Day1', 'Date_Day2'] + all_columns)

In [37]:
#將reshaped_data_df_all有缺失值的row列出，並存成missing_data
missing_data = reshaped_data_df_all[reshaped_data_df_all.isnull().any(axis=1)]
missing_data.shape

missing_data.to_csv(f'./data/preprocess_data/AllVariablesData{name}_na_interpolation.csv',index=False)

In [38]:
#將reshaped_data_df_all中有缺失值的row刪除，並存成csv檔
reshaped_data_df_all.dropna(inplace=True)
reshaped_data_df_all.shape

reshaped_data_df_all.to_csv(f'./data/preprocess_data/AllVariablesData{name}_interpolation.csv',index=False)

將AllVariablesData_na_interpolation整理成可以當測試集的格式

In [39]:
data_na = pd.read_csv(f'./data/preprocess_data/AllVariablesData{name}_na_interpolation.csv')

#把data_na中col4-col363欄位為nan的資料刪除
data_na = data_na.dropna(subset=data_na.columns[3:363])

#把data_na中16:10_2_Power(mW)欄位是nan的資料列出，並叫data_na_1
data_na_1 = data_na[data_na['16:10_2_Power(mW)'].isnull()]

#取data_na_1_onehot前363欄，後240欄刪除
data_na_1_onehot = data_na_1.iloc[:, 0:363]
print("Feature count of new data:", data_na_1_onehot.shape)


Feature count of new data: (204, 363)


In [40]:
# 檢查欄位名稱
column_names = data_na_1_onehot.columns.tolist()
print(column_names)

data_na_1_onehot.to_csv(f'./data/preprocess_data/AllVariablesData{name}_na_interpolation_testset_1.csv', index=False)

['LocationCode', 'Date_Day1', 'Date_Day2', '07:00_1_Pressure(hpa)', '07:00_1_Temperature(°C)', '07:00_1_Humidity(%)', '07:00_1_Sunlight(Lux)', '07:00_1_Power(mW)', '07:10_1_Pressure(hpa)', '07:10_1_Temperature(°C)', '07:10_1_Humidity(%)', '07:10_1_Sunlight(Lux)', '07:10_1_Power(mW)', '07:20_1_Pressure(hpa)', '07:20_1_Temperature(°C)', '07:20_1_Humidity(%)', '07:20_1_Sunlight(Lux)', '07:20_1_Power(mW)', '07:30_1_Pressure(hpa)', '07:30_1_Temperature(°C)', '07:30_1_Humidity(%)', '07:30_1_Sunlight(Lux)', '07:30_1_Power(mW)', '07:40_1_Pressure(hpa)', '07:40_1_Temperature(°C)', '07:40_1_Humidity(%)', '07:40_1_Sunlight(Lux)', '07:40_1_Power(mW)', '07:50_1_Pressure(hpa)', '07:50_1_Temperature(°C)', '07:50_1_Humidity(%)', '07:50_1_Sunlight(Lux)', '07:50_1_Power(mW)', '08:00_1_Pressure(hpa)', '08:00_1_Temperature(°C)', '08:00_1_Humidity(%)', '08:00_1_Sunlight(Lux)', '08:00_1_Power(mW)', '08:10_1_Pressure(hpa)', '08:10_1_Temperature(°C)', '08:10_1_Humidity(%)', '08:10_1_Sunlight(Lux)', '08:10_1_P